In [1]:
import requests
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.nodes import DensePassageRetriever
from transformers import pipeline
requests.get('http://localhost:9200/_cluster/health').json()

{'cluster_name': 'elasticsearch',
 'status': 'yellow',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 10,
 'active_shards': 10,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 3,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 76.92307692307693}

In [2]:
doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)

with open('./meditations.txt', 'r') as f:
    data = f.read()

data = data.split('\n')

data_json = [
    {
        'content': paragraph,
        'meta': {
            'source': 'meditations'
        }
    } for paragraph in data
]

doc_store.write_documents(data_json)
requests.get('http://localhost:9200/aurelius/_count').json()


/home/pancrackhead/anaconda3/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'count': 597,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}}

In [3]:
retriever = DensePassageRetriever(
    document_store=doc_store,
    query_embedding_model='facebook/dpr-question_encoder-single-nq-base',
    passage_embedding_model='facebook/dpr-ctx_encoder-single-nq-base',
    use_gpu=True,
    embed_title=True
)
doc_store.update_embeddings(retriever=retriever)

/home/pancrackhead/anaconda3/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating embeddings:   0%|          | 0/597 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/608 [00:00<?, ? Docs/s]

In [4]:
result = "What do you think about Julius Cesar?"

In [5]:
prompt = result
con = retriever.retrieve(prompt)
# count = 0
ans = []
score = []
print(prompt)
for i in con:
    # print("="*100)
    # count+=1
    # print(count)
    x = i.content
    question_answerer = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')
    # question_answerer = pipeline("question-answering", model='/home/pancrackhead/Desktop/berikc_stable/fine_tuned_bert')
    context = x
    result = question_answerer(question=prompt, context=context)
    ans.append(result['answer'])
    score.append(result['score'])
    
    # print(
    # f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")
    # print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}")
# print(result['answer'][0])

score, ans = zip(*sorted(zip(score, ans), reverse=True))

for i in score:
    print(i)
print(ans)


What do you think about Julius Cesar?
0.4583497643470764
0.1835387945175171
0.132802814245224
0.10407187789678574
0.09345057606697083
0.052504267543554306
0.04106731712818146
0.03497106954455376
0.03272003307938576
0.01962636224925518
('the whole court is dead', 'What soul then has skill and knowledge? That which knows beginning and end', 'those among us who are called Patricians are rather deficient in paternal affection', 'benevolent to his own kind', 'I have no leisure', 'how little he was satisfied', 'benevolent disposition', 'think where are they all now', 'if they have not determined about me individually', 'I observed no concealment of his opinions with respect to those whom he condemned')
